In [14]:
from z3 import *
from itertools import product

This is an example of how to use the optimizer in Z3, based on [this documentation snippet](https://www.philipzucker.com/z3-rise4fun/optimization.html).

The problem to solve is placing VMs on different servers, based on (1) VM space requirement, (2) server cost, and (3) how much space each server has. In particular, the goal is to minimize:

  1. The number of servers used
  2. The operation cost

Consider VM 1, VM 2, and VM 3, that need 100, 50, and 15 GB space respectively:

In [15]:
class VM:
    id: str
    space_needed: int
    def __init__(self, id: str, space_needed: int):
        self.id = id
        self.space_needed = space_needed

VM1, VM2, VM3 = VM("1", 100), VM("2", 50), VM("3", 15)

Next, consider three servers that have 100, 75, and 200 GB of space respectively. They also have a daily fixed cost of \\$10, \\$5, and \\$20. 

In [16]:
class Server:
    id: str
    space: int
    cost: int
    def __init__(self, id: str, space: int, cost: int):
        self.id = id
        self.space = space
        self.cost = cost
        
S1, S2, S3 = Server("1", 100, 10), Server("2", 75, 5), Server("3", 200, 20)

The variable $x_{ij}$ will represent that VM $i$ is placed on server $j$. The variable $y_i$ will denote that server $i$ is in use.

In [17]:
def place_VM(vm: VM, server: Server):
    return Int(f"x{vm.id}{server.id}")

def in_use(server: Server):
    return Int(f"y{server.id}")

Z3 uses the `Optimize` class to represent the constraint solver.

In [18]:
o = Optimize()

The next step is to add all the constraints to the optimizer instance `o`.

In [19]:
# The integers representing placement and use should not be negative.
for vm, server in product([VM1, VM2, VM3], [S1, S2, S3]):
    o.add(place_VM(vm, server) >= 0)
    
# A server is either in use, or not in use.
for server in [S1, S2, S3]:
    o.add(in_use(server) <= 1)

# VMs can only be on one server at a time
for vm in [VM1, VM2, VM3]:
    o.add(place_VM(vm, S1) + place_VM(vm, S2) + place_VM(vm, S3) == 1)
    
# placing a VM on a server makes it "in use"
for server in [S1, S2, S3]:
    o.add(And(in_use(server) >= place_VM(VM1, server), in_use(server) >= place_VM(VM2, server), in_use(server) >= place_VM(VM3, server)))

The above are mainly physical constraints; below are the specific constraints of this problem, such as server space, server cost, and space required by each VM: 

In [20]:
for server in [S1, S2, S3]:
    o.add(Sum([vm.space_needed * place_VM(vm, server) for vm in [VM1, VM2, VM3]]) <= server.space * in_use(server))

For this specific problem, the loop above unrolls into the below constraints:

In [21]:
# VM1 needs 100 GB, VM2 needs 50 GB, VM3 needs 15GB
# server 1 has 100 GB of disk space: 100*place_VM(1, 1) + 50*place_VM(2, 1) + 15*place_VM(3, 1) <= 100*in_use(1)
# server 2 has 75 GB: 100*place_VM(1, 2) + 50*place_VM(2, 2) + 15*place_VM(3, 2) <= 75*in_use(2)
# server 3 has 200 GB: 100*place_VM(1, 3) + 50*place_VM(2, 3) + 15*place_VM(3, 3) <= 200*in_use(3)

Finally, there are two minimization goals in this problem: (1) number of servers, and (2) cost.

In [22]:
# minimize number of servers used
o.minimize(in_use(S1) + in_use(S2) + in_use(S3))

# minimize operation cost
o.minimize(S1.cost*in_use(S1) + S2.cost*in_use(S2) + S3.cost*in_use(S3))

To obtain a solution, `check` needs to be called first. This will be either `sat` or `unsat`. Most of the time `sat` is expected, but the result might be `unsat` if there is no solution to the problem under the given constraints. 
Then, the solution can be obtained using `model`:

In [23]:
o.check()
[(s, o.model()[s]) for s in o.model()]

[(x33, 1),
 (x13, 1),
 (y2, 0),
 (y3, 1),
 (y1, 0),
 (x32, 0),
 (x22, 0),
 (x12, 0),
 (x23, 1),
 (x31, 0),
 (x21, 0),
 (x11, 0)]

The code below maps the variables back to VM server assignments:

In [24]:
for sol in o.model():
    if o.model()[sol] == 1:
        if len(sol.name()) == 3:
            print(f"allocate VM {sol.name()[1]} on server {sol.name()[2]}")

allocate VM 3 on server 3
allocate VM 1 on server 3
allocate VM 2 on server 3


Even though server 3 is the most expensive (\\$20), it has enough space to host all VMs (`100 + 50 + 15 = 165 < 200`). Two instances of S1 wouldn't be any cheaper, since it has a cost of \\$10. It would be cheaper to have server 1 and server 2, or, three instances of server 2; however, that doesn't minimize the total number of servers enough, such that the reduced cost justifies more servers.